In [1]:
from NER_processing import nlp, disease_service, chemical_service, genetic_service
import pysolr

In [44]:
class CORD19Processor:
    def __init__(self,window_size):
        self.covid_articles = pysolr.Solr('http://librairy.linkeddata.es/data/covid', timeout=10)
        self.covid_paragraphs = pysolr.Solr('http://librairy.linkeddata.es/data/covid-paragraphs', timeout=10)
        self.window_size = window_size
        self.annotated_paragraphs = self.annotate_paragraphs()
    
    
    def annotate_paragraphs(self):
        counter = 0
        completed = False
        cursor = "*"
        i=0
        j=0
        processed_paragraphs = []
        while (not completed):
            old_counter = counter
            try:
                results = self.covid_paragraphs.search(q="*:*",cursorMark=cursor, rows=self.window_size,sort="id asc")
                cursor = results.nextCursorMark
                counter += len(results)
                paragraphs = []
                print(len(results))
                for result in results:
                    paragraph = {}
                    if ('section_s' in result):
                        paragraph['section_s'] = result['section_s']
                    if ('id' in result):
                        paragraph['id'] = result['id']
                    if ('article_id_s' in result):
                        paragraph['article_id_s'] = result['article_id_s']
                    if ('text_t' in result):
                        paragraph['text_t'] = result['text_t']
                    if ('size_i' in result):
                        paragraph['size_i'] = result['size_i']
                    if ('name_s' in result):
                        paragraph['name_s'] = result['name_s']
                    paragraphs.append(paragraph)
                    j+=1
                    if j%100000 == 0:
                        print(j, 'paragraphs retrieved')
                print(len(paragraphs),'paragraphs retrieved')
                if (old_counter == counter):
                    print('Final')
                    break
        
                for paragraph in paragraphs:
                    doc = nlp(paragraph['text_t'])
                    paragraph['disease_ents'] = [f for f in doc.ents if f.label_ == 'DISEASE']
                    paragraph['chemical_ents'] = [f for f in doc.ents if f.label_ == 'CHEMICAL']
                    paragraph['genetic_ents'] = [f for f in doc.ents if f.label_ == 'GENETIC']
                    i+=1
                    if i%10000 == 0:
                        print(i, 'paragraphs processed')
                    processed_paragraphs.append(paragraph)
               
                completed = True
            except Exception as e:
                print(repr(e))
                        
        return paragraph

        
            
            
            
            

In [45]:
CORD19Processor(window_size=200)

2103891
100000 paragraphs retrieved
200000 paragraphs retrieved
300000 paragraphs retrieved
400000 paragraphs retrieved
500000 paragraphs retrieved
600000 paragraphs retrieved
700000 paragraphs retrieved
800000 paragraphs retrieved
900000 paragraphs retrieved
1000000 paragraphs retrieved
1100000 paragraphs retrieved
1200000 paragraphs retrieved
1300000 paragraphs retrieved
1400000 paragraphs retrieved
1500000 paragraphs retrieved
1600000 paragraphs retrieved
1700000 paragraphs retrieved
1800000 paragraphs retrieved
1900000 paragraphs retrieved
2000000 paragraphs retrieved
2100000 paragraphs retrieved
2103891 paragraphs retrieved
100 paragraphs processed
{'section_s': 'background', 'id': '0002d368d74a23c546db9bc6098c37fe', 'article_id_s': '744f2ba295872a8f27eae0204ba16b3afb3bd7c9', 'text_t': 'Aseptic meningitis, encephalitis, and myelitis are inflammatory conditions of the central nervous system (CNS) (meninges, brain, and spinal cord, respectively). Disease is caused by a variety of in

KeyboardInterrupt: 

In [4]:
process_cord19()

1000 paragraphs retrieved
100 paragraphs processed
{'section_s': 'subunit vaccines', 'id': '43c7274ceb6399f07a7e0a52bbcd30ba', 'article_id_s': 'a9754e66d1d118157221031daab60604bab2ad14', 'text_t': 'Both MERS-CoV and SARS-CoV RBDs consist of a core and a receptor-binding subdomain. They share a high degree of structural similarity in the core subdomains, but the receptor-binding subdomains are notably divergent [32] . Because of variation of the receptor-binding subdomain region within subgroups of the same CoV or across different CoV groups, it is advisable to rationally design MERS subunit vaccines using chimeric S protein containing several neutralizing epitopes from divergent subgroups. This approach will provide a strategic platform for the rational design of subunit vaccines against future emerging CoVs by focusing on the chimeric S protein containing neutralizing epitopes from multiple virus strains across different subgroups [94] .', 'size_i': 754, 'disease_ents': [MERS-CoV, SAR

In [4]:
doc = nlp('pneumonia')

0
0
